# Introduction to Computer Vision and OpenCV:

Understand the fundamentals of computer vision and its applications.
Learn the basics of the OpenCV library and its core functions.
Setup your development environment with OpenCv
Load and display images using OpenCV

In [1]:
import cv2
import numpy as np

In [ ]:
%pwd

: 

In [ ]:
cv2.__version__

: 

In [ ]:
img = cv2.imread('./cat.jpg')

: 

In [ ]:
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

: 

# Image Processing Techniques using OpenCV:

Implement basic image processing techniques such as resizing, cropping, and color manipulation.
Apply various filters and enhancements to images.

In [ ]:
x_cord = 0
y_cord = 0

def mouse_callback(event, x, y, flats, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x,y), radius=3, color=(0, 0, 255), thickness=2)
        cv2.putText(img, f'x:{x}', (x+10,y-10) , 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, 
                    (0,0,0), 2)
        cv2.putText(img, f'y:{y}', (x+10,y+10) , 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    (0,0,0), 2)
        cv2.imshow('Image', img)
        print(f"Coordinates: {x}, {y}")

# Create a window and set the mouse callback function
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', mouse_callback)
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

: 

In [ ]:
def show_img(imgs):
    for k, img in imgs.items():
        cv2.imshow(k, img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

: 

In [ ]:
cat_face = img[18:356, 103:416]
resized = cv2.resize(img, (100,100))

: 

In [ ]:
cv2.imshow('img', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

: 

In [ ]:
blurred_img = cv2.GaussianBlur(img, (9,9), 0)

: 

In [ ]:
kernel = np.ones((5,5), np.uint8)

: 

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

: 

In [ ]:
_, otsu_thresholding = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

: 

In [ ]:
canny = cv2.Canny(img_gray,50,300)

: 

In [ ]:
opening = cv2.morphologyEx(otsu_thresholding, cv2.MORPH_OPEN, kernel)

: 

In [ ]:
inverted_img = 255 - img

: 

In [ ]:
show_img({'opening':opening,
          'otsu_thresholding':otsu_thresholding,
          'Gaussian':blurred_img,
          'invert':inverted_img})


: 

# Object Detection and Tracking:

Implement object detection using pre-trained deep learning models (e.g., YOLO or SSD).
Develop real-time object tracking using techniques like the Kalman filter or Mean-Shift algorithm.

In [ ]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

 

# Load YOLO model and COCO class names
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
classes = []
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

 

# Create a Tkinter GUI
root = tk.Tk()
root.title('YOLO Object Detection App')

 

# Function to open an image using a file dialog
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        detect_objects(image)

 

# Function to display an OpenCV image in the GUI
def display_image(cv_image):
    pil_image = Image.fromarray(cv_image)
    tk_image = ImageTk.PhotoImage(image=pil_image)
    label.config(image=tk_image)
    label.image = tk_image

 

# Function to perform object detection
def detect_objects(cv_image):
    height, width = cv_image.shape[:2]

 

    # Prepare input image for YOLO model
    blob = cv2.dnn.blobFromImage(cv_image, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

 

    # Get detection results
    outs = net.forward(net.getUnconnectedOutLayersNames())

 

    # Process detection results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = scores.argmax()
            confidence = scores[class_id]
            if confidence > 0.7:  # Set a confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

 

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

 

                # Draw bounding box and class label
                cv2.rectangle(cv_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
                label = f'{classes[class_id]}: {confidence:.2f}'
                cv2.putText(cv_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

 

    display_image(cv_image)

 

# Create buttons for opening an image
open_button = tk.Button(root, text="Open Image", command=open_image)

 

# Create an image display label
label = tk.Label(root)
label.pack()

 

# Place buttons in the GUI
open_button.pack()

 

root.mainloop()


: 

In [ ]:
resized = cv2.resize(img, (416,416))

scalefactor = 1.0/255.0
size = (416,416)
mean = (0,0) # Mean values for image Normalisation
swapRB = True # Swap red and blue channels for image preprocessing
crop = False # Do not crop the image during blob creation

blob = cv2.dnn.blobFromImage(resized,
                            scalefactor,
                            size,
                            mean,
                            swapRB=swapRB,
                            crop=crop)

net.setInput(blob)
output_layers = net.getUnconnectedOutLayersNames()
outs = net.forward(output_layers)

# tracker = cv2.TrackerKCF_create()
tracker = cv2.legacy_TrackerMOSSE.create()

bbox = cv2.selectROI(img, False)

tracker.init(img, bbox)

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)

: 

In [ ]:
outs

: 

In [ ]:
show_img({'':resized})

: 

In [ ]:
cv2.__version__

: 

# Feature Detection and Matching:

Explore feature detection methods such as SIFT or ORB.
Implement feature matching to establish correspondence between images.

In [ ]:
import cv2
import numpy as np

img1 = cv2.imread('./cat.jpg')
img2 = cv2.imread('./cat2.jpg')

# Create a ORB detector
orb = cv2.ORB_create()

# Find key Points and descriptors
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

# Initialize Brute force matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match Descriptors 
matches = bf.match(descriptors1, descriptors2)

matching_results = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:30], None, flags=2)

: 

* im1 - The source image
* keypoints1 - list of key points detected in the first image
* im2 - second image where matches will be viualized
* keypoints2 - List of keypoints detected in second image
* matches[:20] - a subset of matches to be draw
* None - Optional mask image, if provided, only the key points that are within the mask will be used in the matching visualisation 
* flats = 2 - the flag that specifies how to draw the matches


In [ ]:
show_img({'':matching_results})

: 

In [ ]:
show_img({'':descriptors2})

: 

# Activity 5: Image Filtering and Transformation:

 Apply advanced image filtering techniques, such as Gaussian blur and edge detection.
 Explore geometric transformations like rotation, scaling, and perspective transformation.

In [ ]:
gaussian = cv2.GaussianBlur(img1, (11,11), 9)

: 

In [ ]:
gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

: 

In [ ]:
edges = cv2.Canny(gray,100,200)

: 

In [ ]:
# Rotation
angle = 30
rows, cols = img1.shape[:2]
rotation_matrix = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
rotated_matrix = cv2.warpAffine(img1, rotation_matrix, (cols, rows))


: 

In [ ]:
show_img({'':gaussian,
          'cann':edges})

: 

In [ ]:
show_img({'':rotated_matrix})

: 

In [ ]:
scale_factor = 0.5
scaled = cv2.resize(img1, None, fx=scale_factor, fy=scale_factor)


: 

In [ ]:
show_img({'':scaled})

: 

In [ ]:
# Perspective transformation
pts1 = np.float32([[50,50], [200,50], [50, 200], [200, 200]])
pts2 = np.float32([[0,0], [250,0], [0, 250], [250, 250]])

perspective_matrix = cv2.getPerspectiveTransform(pts1, pts2)
perspective_transformed = cv2.warpPerspective(img1, perspective_matrix, (250,250))

: 

In [ ]:
show_img({'':perspective_transformed})

: 

# Activity 6: Advanced Computer Vision Techniques:
 Dive into advanced techniques like image segmentation and contour detection.

 Implement optical flow for motion analysis

In [ ]:
import cv2 
import numpy as np

img = cv2.imread('cat.jpg', )
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

: 

In [ ]:
edges = cv2.Canny(gray, 100, 200)

: 

In [ ]:
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

: 

In [ ]:
contour_images = img.copy()

: 

In [ ]:
# Draw and display contours 
cv2.drawContours(contour_images, contours, -1, (0, 255, 0), 2)

: 

In [ ]:
show_img({'':contour_images})

: 

## Implement optical flow for motion analysis.

In [ ]:
import cv2 
import numpy as np

# Capture video feed 
cap = cv2.VideoCapture(0)

# Read the frame
ret1, frame1 = cap.read()
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    ret, frame2 = cap.read()
    current_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Draw motion vectors
    motion_image = np.copy(frame2)
    for y in range(0, motion_image.shape[0], 10):
        for x in range(0, motion_image.shape[1], 10):
            dx, dy = flow[y, x]
            cv2.arrowedLine(motion_image, (x, y), (int(x+dx), int(y+dy)), (0, 0, 255), 1)
    
    prev_gray = current_gray

    # cv2.imshow('Video', frame1)
    # cv2.imshow('Gray Video', prev_gray)
    cv2.imshow('Optical Flow', motion_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


    

: 

In [ ]:
# UI

: 

In [ ]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk




# Create a Tinker GUI
root = tk.Tk()
root.title('Open CV Image Processing APP') 




# Function to open a image
def oepn_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        display_image(image)

# Function to display image
def display_image(image):
    pil_image = Image.fromarray(image)
    tk_image = ImageTk.PhotoImage(image = pil_image)
    label.config(image = tk_image)
    label.image = tk_image


# Create a button
open_button = tk.Button(root, text = 'Open Image', command = oepn_image)

# Create an Image display label 
label = tk.Label(root)
label.pack()

# Plaace the button
open_button.pack()

root.mainloop()

: 

In [ ]:
# from tkmacosx import Label

import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

 

#Create a Tinker GUI
root = tk.Tk()
root.title("OpenCV Image Processing APP")

 

#Function to open a image
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = cv2.imread(file_path)
        img_cvt = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        display_image(img_cvt)

 

#Function to display image
def display_image(image):
    pil_image = Image.fromarray(image)
    tk_image = ImageTk.PhotoImage(image = pil_image)
    label.config(image = tk_image)
    label.image = tk_image
    label.pack()

def convert_to_grayscale():
    
    pass


def edge_detect():
    pass

#Create a button 
open_button = tk.Button(root , text = "Open Image" , command = open_image)
grayscale_button = tk.Button(root, text='Convert to grayScale', command=convert_to_grayscale)
edge_button = tk.Button(root, text='Edge Detection', command=edge_detect)

#Create an Image display label
label = tk.Label(root)
label.pack()

 

#Place the button
open_button.pack()
grayscale_button.pack()
edge_button.pack()
 

root.mainloop()

: 

In [1]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

 

# Load YOLO model and COCO class names
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
classes = []
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

 

# Create a Tkinter GUI
root = tk.Tk()
root.title('YOLO Object Detection App')

 

# Function to open an image using a file dialog
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        detect_objects(image)

def convert_to_grayscale():
    gray_image = cv2.cvtColor(my_image , cv2.COLOR_BGR2GRAY)
    display_image(gray_image)

 

def edge_detect():
    edges = cv2.Canny(my_image , 100 , 200)
    display_image(edges)

 

#Create a button 


# Function to display an OpenCV image in the GUI
def display_image(cv_image):
    pil_image = Image.fromarray(cv_image)
    tk_image = ImageTk.PhotoImage(image=pil_image)
    label.config(image=tk_image)
    label.image = tk_image

 

# Function to perform object detection
def detect_objects(cv_image):
    height, width = cv_image.shape[:2]

 

    # Prepare input image for YOLO model
    blob = cv2.dnn.blobFromImage(cv_image, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

 

    # Get detection results
    outs = net.forward(net.getUnconnectedOutLayersNames())

 

    # Process detection results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = scores.argmax()
            confidence = scores[class_id]
            if confidence > 0.7:  # Set a confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

 

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

 

                # Draw bounding box and class label
                cv2.rectangle(cv_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
                label = f'{classes[class_id]}: {confidence:.2f}'
                cv2.putText(cv_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

 

    display_image(cv_image)

 

# Create buttons for opening an image
open_button = tk.Button(root, text="Open Image", command=open_image)
grayscale_button = tk.Button(root , text = "Convert to grayScale" , command = convert_to_grayscale)
edge_button = tk.Button(root , text = "Edge Detection" , command = edge_detect)
 

# Create an image display label
label = tk.Label(root)
label.pack()

 

# Place buttons in the GUI
open_button.pack()
grayscale_button.pack()
edge_button.pack()

 

root.mainloop()
 

2023-08-22 01:41:38.026 python3.10[34484:178590] +[CATransaction synchronize] called within transaction
2023-08-22 01:41:38.185 python3.10[34484:178590] +[CATransaction synchronize] called within transaction


In [4]:
# First things, first. Import the wxPython package.
import wx

# Next, create an application object.
app = wx.App()

img = None

# Then a frame.
frm = wx.Frame(None, title="Hello World")

def onOpen(event):
    print('test on open')

    # otherwise ask the user what new file to open
    with wx.FileDialog(None, "Open XYZ file",
                       style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) as fileDialog:

        if fileDialog.ShowModal() == wx.ID_CANCEL:
            return     # the user changed their mind

        # Proceed loading the file chosen by the user
        pathname = fileDialog.GetPath()






button = wx.Button(frm, wx.ID_ANY, 'Test', (10, 10))
button.Bind(wx.EVT_BUTTON, onOpen)

# Show it.
frm.Show()

# Start the event loop.
app.MainLoop()

PyNoAppError: The wx.App object must be created first!

In [ ]:
import wx
import cv2
from PIL import Image

class Frame(wx.Frame):

    def __init__(self, parent):
        super().__init__(parent)
        
        # self.MainPanel = wx.Panel(self)

        fgs = wx.FlexGridSizer(3, 2, 10, 10)
        fgs.AddGrowableRow(0)

        self.staticbitmap = wx.StaticBitmap(self)
        fgs.Add(self.staticbitmap, 0, wx.EXPAND)

        fgs.RemoveGrowableRow(0)
        browse = wx.Button(self, label='Browse')
        browse.Bind(wx.EVT_BUTTON, self.OnBrowse)
        fgs.Add(browse,1, flag=wx.LEFT)

        to_gray = wx.Button(self, label='To Gray')
        to_gray.Bind(wx.EVT_BUTTON, self.ToGray)
        fgs.Add(to_gray,2, flag=wx.RIGHT)

        detect_edge = wx.Button(self, label='Detect Edge')
        detect_edge.Bind(wx.EVT_BUTTON, self.DetectEdges)
        fgs.Add(detect_edge, 2, 2)
        
        self.SetSizer(fgs)

    def OnBrowse(self, event):
        wildcard = 'PNG files (*.png)|*.png|BMP files (*.bmp)|*.bmp|JPEG files (*.jpg)|*.jpg'
        openFileDialog = wx.FileDialog(self, "Open", "", "", wildcard,
                                       wx.FD_OPEN | wx.FD_FILE_MUST_EXIST)
        openFileDialog.ShowModal()
        file_path = openFileDialog.GetPath()
        self.img =  cv2.imread(file_path)
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        self.UpdateImage()
        event.Skip()

    def ToGray(self, event):
        self.img = cv2.cvtColor(self.img, cv2.COLOR_RGB2GRAY)

        # Because wx buffer object need image to be RGB format
        self.img = cv2.cvtColor(self.img, cv2.COLOR_GRAY2RGB)
        self.UpdateImage()
        event.Skip()

    def UpdateImage(self):
        bmp = wx.BitmapFromBuffer(self.img.shape[1], self.img.shape[0], self.img)
        self.staticbitmap.SetBitmap(bmp)

    def DetectEdges(self, event):
        edges = cv2.Canny(self.img , 100 , 200)
        edges = Image.fromarray(edges)
        edges_img = wx.Image(edges.size[0], edges.size[1])
        edges_img.SetData(pil_image.convert("RGB").tobytes())
        
        bitmap = wx.Bitmap(edges_img)
        self.staticbitmap.SetBitmap(bmp)
        event.Skip()


app = wx.App()
frame = Frame(None)
frame.Show()
app.MainLoop()